In [5]:
%matplotlib tk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plot_interactions import on_hover, on_click


In [6]:
# Check which Python interpreter and kernel you're using
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


Python executable: /home/lcb/Documents/2026_01_21_shim_tests/.venv/bin/python
Python version: 3.14.2 (main, Dec  5 2025, 00:00:00) [GCC 15.2.1 20251111 (Red Hat 15.2.1-4)]
Numpy version: 2.4.1
Pandas version: 3.0.0


In [7]:
# basename = "5-05_all_ch_again"
# basename = "5-03_ch_1_ramp_twice"
# basename = "bench_dr_4ms"
basename = "new_bench_dr_20MHz"


In [8]:
dfs = [pd.DataFrame() for _ in range(4)]

# Load the trigger file to determine number of slices
trig_df = pd.read_csv(f"data/{basename}_trig.csv", sep=r"\s+", header=None)
num_slices = len(trig_df)

# Load the files
for i in range(4):
  dfs[i] = pd.read_csv(f"data/{basename}_bd_{i}.csv", sep=r"\s+", header=None)

# Initialize the array
adc_array = dfs[0].to_numpy()

# Append each subsequent DataFrame to the array (in the second dimension)
for i in range(1, 4):
  adc_array = np.concatenate((adc_array, dfs[i].to_numpy()), axis=1)

# Convert ADC values to amps (-2^15 to 2^15 -> -5.0 to 5.0 A)
adc_array = adc_array * (5.0 / 32768)

# Calculate slice_size based on total samples and number of slices
total_samples = adc_array.shape[0]
slice_size = total_samples // num_slices
num_channels = adc_array.shape[1]

# Calculate global min/max for consistent y-axis limits
global_min = np.min(adc_array)
global_max = np.max(adc_array)
y_range = global_max - global_min
padding = y_range * 0.05
global_ylim = (global_min - padding, global_max + padding)

print(f"Detected {num_slices} slices from trigger file")
print(f"Total samples: {total_samples}, Slice size: {slice_size}")
print(f"Number of channels: {num_channels}")
print(f"Global y-axis limits: {global_ylim[0]:.3f} to {global_ylim[1]:.3f} A")

time_slice = np.linspace(0, 10e-3, slice_size, endpoint=False)

# Interactive toggle plot: starts with slices for channel 0, click to switch modes
fig, ax = plt.subplots(figsize=(14, 7))

# State tracking
state = {
    'mode': 'slices',  # 'slices' or 'channels'
    'channel_idx': 0,  # Current channel when in slices mode
    'slice_idx': 0,    # Current slice when in channels mode
    'lines': [],
    'annotation': None,
    'highlighted_line': [None]
}

def update_plot():
    """Redraw the plot based on current state"""
    ax.clear()
    state['lines'] = []
    
    if state['mode'] == 'slices':
        # Show all slices for current channel
        for slice_idx in range(num_slices):
            slice_data = adc_array[slice_idx*slice_size:(slice_idx+1)*slice_size, state['channel_idx']]
            line, = ax.plot(time_slice, slice_data, alpha=0.5, linewidth=2)
            state['lines'].append(line)
        
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Current (A)')
        ax.set_title(f"Channel {state['channel_idx']} - All Slices (Hover to identify, Click to view that slice's channels)")
        ax.grid(True)
        
    else:  # mode == 'channels'
        # Show all channels for current slice
        slice_data = adc_array[state['slice_idx']*slice_size:(state['slice_idx']+1)*slice_size, :]
        for ch_idx in range(num_channels):
            line, = ax.plot(time_slice, slice_data[:, ch_idx], alpha=0.5, linewidth=2)
            state['lines'].append(line)
        
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Current (A)')
        ax.set_title(f"Slice {state['slice_idx']} - All Channels (Hover to identify, Click to view that channel's slices)")
        ax.grid(True)
    
    # Always set to global y-axis limits
    ax.set_ylim(global_ylim)
    
    # Create new annotation
    state['annotation'] = ax.annotate('', xy=(0,0), xytext=(20,20), textcoords='offset points',
                                      bbox=dict(boxstyle='round', fc='yellow', alpha=0.9),
                                      arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                                      visible=False, fontsize=12, weight='bold')
    state['highlighted_line'][0] = None
    fig.canvas.draw_idle()

def on_hover_toggle(event):
    """Handle hover events"""
    if event.inaxes != ax:
        if state['annotation']:
            state['annotation'].set_visible(False)
        if state['highlighted_line'][0] is not None:
            state['highlighted_line'][0].set_linewidth(2)
            state['highlighted_line'][0].set_alpha(0.5)
            state['highlighted_line'][0] = None
        fig.canvas.draw_idle()
        return
    
    # Check if hovering over any line
    found = False
    for i, line in enumerate(state['lines']):
        if line.contains(event)[0]:
            found = True
            # Reset previous highlight
            if state['highlighted_line'][0] is not None and state['highlighted_line'][0] != line:
                state['highlighted_line'][0].set_linewidth(2)
                state['highlighted_line'][0].set_alpha(0.5)
            
            # Highlight current line
            line.set_linewidth(3)
            line.set_alpha(1.0)
            state['highlighted_line'][0] = line
            
            # Update annotation
            if state['mode'] == 'slices':
                label = f'Slice {i}'
            else:
                label = f'Channel {i}'
            state['annotation'].set_text(label)
            state['annotation'].xy = (event.xdata, event.ydata)
            state['annotation'].set_visible(True)
            fig.canvas.draw_idle()
            break
    
    if not found:
        state['annotation'].set_visible(False)
        if state['highlighted_line'][0] is not None:
            state['highlighted_line'][0].set_linewidth(2)
            state['highlighted_line'][0].set_alpha(0.5)
            state['highlighted_line'][0] = None
        fig.canvas.draw_idle()

def on_click_toggle(event):
    """Handle click events to toggle modes"""
    if event.inaxes != ax:
        return
    
    # Check if clicking on any line
    for i, line in enumerate(state['lines']):
        if line.contains(event)[0]:
            # Toggle mode and update indices
            if state['mode'] == 'slices':
                # Switch to channels mode for clicked slice
                state['mode'] = 'channels'
                state['slice_idx'] = i
            else:
                # Switch to slices mode for clicked channel
                state['mode'] = 'slices'
                state['channel_idx'] = i
            
            update_plot()
            break

# Initial plot
update_plot()

# Connect events
fig.canvas.mpl_connect('motion_notify_event', on_hover_toggle)
fig.canvas.mpl_connect('button_press_event', on_click_toggle)

plt.show()


Detected 320 slices from trigger file
Total samples: 64000, Slice size: 200
Number of channels: 32
Global y-axis limits: -1.640 to 1.495 A
